In [ ]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [1]:
import requests
import json
from pprint import pp

In [2]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('openai_api_key')

In [3]:
def get_completion(messages, model="gpt-4.1-nano", reasoning_effort=None):
  payload = { "model": model, "messages": messages }

  if reasoning_effort:
    payload["reasoning_effort"] = reasoning_effort

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj # obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

## 使用推理模型

注意: o 系列不支援 temperature 參數

In [5]:
user_message = "房間裡有三盞燈外面有三個開關。你站在房間外，看不到燈的狀態。你可以隨意操作開關（開或關），也可以進房間檢查一次燈的情形，但只能進去一次。請問：你該怎麼操作，才能判斷每個開關對應哪一盞燈？"

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response2 = get_completion(messages, model="o4-mini", reasoning_effort="low") # medium, high
response2



{'id': 'chatcmpl-Bt6PuwWhhrqulbsmsJFaGlNcgjjv7',
 'object': 'chat.completion',
 'created': 1752472406,
 'model': 'o4-mini-2025-04-16',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '你可以按下面這個順序操作：  \n\n1. 把開關A（以下為標示，實際可任意命名）打開，等約５分鐘，讓燈泡充分發熱。  \n2. ５分鐘後，把開關A關掉，立即把開關B打開，然後立刻進房間。  \n\n進到房間後，你會看到三盞燈的情況：  \n- 那盞「亮着」的燈，必然是開關B控制的。  \n- 那盞「暗着但摸起來燙手」的燈，是剛才開了５分鐘又關掉的開關A控制的。  \n- 剩下那盞「既不亮也不燙」的燈，就是開關C控制的。  \n\n如此，一次進房就能對應出三個開關與三盞燈的對應關係。',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 99,
  'completion_tokens': 291,
  'total_tokens': 390,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 64,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [14]:
print(response2["choices"][0]["message"]["content"])

你可以按下面這個順序操作：  

1. 把開關A（以下為標示，實際可任意命名）打開，等約５分鐘，讓燈泡充分發熱。  
2. ５分鐘後，把開關A關掉，立即把開關B打開，然後立刻進房間。  

進到房間後，你會看到三盞燈的情況：  
- 那盞「亮着」的燈，必然是開關B控制的。  
- 那盞「暗着但摸起來燙手」的燈，是剛才開了５分鐘又關掉的開關A控制的。  
- 剩下那盞「既不亮也不燙」的燈，就是開關C控制的。  

如此，一次進房就能對應出三個開關與三盞燈的對應關係。


## OpenAI 不給你看完整的 raw reasoning tokens，但 reasoning_tokens 也是要算錢的

* ChatGPT 上的那個 CoT 過程是摘要過的，非原本的 CoT tokens
* API 說也有開放推理摘要了，但需要組織認證以及用新的 Responses API https://platform.openai.com/docs/guides/reasoning?api-mode=responses#reasoning-summaries


## 試試有公開 thinking 過程的模型: DeepSeek R1

In [ ]:
# from google.colab import userdata
# deepseek_api_key = userdata.get('deepseek_api_key')


In [22]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
deepseek_api_key = os.getenv('deepseek_api_key')


In [19]:
def get_deepseek_completion(messages, model="deepseek-reasoner"):
  payload = { "model": model, "messages": messages }

  headers = { "Authorization": f'Bearer {deepseek_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.deepseek.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj # obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

In [23]:
user_message = "房間裡有三盞燈外面有三個開關。你站在房間外，看不到燈的狀態。你可以隨意操作開關（開或關），也可以進房間檢查一次燈的情形，但只能進去一次。請問：你該怎麼操作，才能判斷每個開關對應哪一盞燈？"

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response6 = get_deepseek_completion(messages)
response6

{'id': '4ae13271-103d-4993-8a7e-de4c301f4bd8',
 'object': 'chat.completion',
 'created': 1752473777,
 'model': 'deepseek-reasoner',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '要判斷每個開關對應哪一盞燈，你只能進房間一次，因此需要利用燈泡的狀態（亮、滅但熱、滅但冷）來區分。燈泡在亮過後會變熱，冷卻需要時間，這提供了第三種狀態（滅但熱）。以下是操作步驟：\n\n1. **準備階段（在房間外操作開關）**：\n   - 將三個開關標記為開關 A、開關 B 和開關 C（以便區分）。\n   - 先確保所有開關都處於「關」的位置，等待約 5 分鐘，讓所有燈泡完全冷卻（這確保初始狀態一致）。\n\n2. **操作開關順序**：\n   - 打開開關 A，保持開啟狀態約 5 分鐘（讓對應的燈泡變熱）。\n   - 5 分鐘後，關掉開關 A。\n   - 立即打開開關 B（不要等待）。\n   - 保持開關 C 始終關閉（從未動過）。\n   - 操作完成後，立即進房間檢查（避免燈泡冷卻）。\n\n3. **進房間檢查燈泡狀態**：\n   - 觀察燈泡的三種狀態：\n     - **亮的燈**：對應開關 B（因為開關 B 被打開，燈泡亮著）。\n     - **滅但熱的燈**（用手觸摸確認溫度）：對應開關 A（因為開關 A 被打開過一段時間，燈泡變熱，然後關掉）。\n     - **滅但冷的燈**：對應開關 C（因為開關 C 從未開啟，燈泡未亮過，保持冷卻）。\n\n### 為什麼這樣有效？\n- 開關 B 控制的燈泡是亮的，因為你剛打開開關 B。\n- 開關 A 控制的燈泡是滅但熱的，因為它被打開過 5 分鐘，熱量尚未散失。\n- 開關 C 控制的燈泡是滅但冷的，因為它從未被開啟，沒有熱量產生。\n- 這樣，三種狀態（亮、滅熱、滅冷）正好對應三個開關，讓你只進房間一次就能判斷所有對應關係。\n\n### 注意事項：\n- **時間控制**：等待 5 分鐘是為了讓燈泡充分加熱（適用於白熾燈等會發熱的燈泡；如果是 LED 燈，可能不適

In [24]:
print(response6["choices"][0]["message"]["reasoning_content"])

首先，問題是：房間裡有三盞燈，外面有三個開關。我看不到燈的狀態，只能操作開關。我可以隨意開關這些開關，然後進房間檢查一次燈的情形。目標是判斷每個開關對應哪一盞燈。

關鍵是：我只能進房間一次，所以我必須在進房間之前，通過操作開關來創造一些狀態，讓我能從燈的狀態推斷出哪個開關控制哪盞燈。

燈的狀態只有兩種：亮或滅。但因為我只能進一次房間，我需要更多資訊來區分三個開關。

所以，我必須利用時間或溫度來創造第三種狀態。燈泡在關掉後，如果剛亮過，會是熱的；如果沒亮過，是冷的。這樣，燈泡就有三種狀態：亮、滅但熱、滅但冷。

這是一個經典謎題的標準解法。讓我回想一下。

步驟應該是：

1. 先打開第一個開關，讓它開著一段時間。

2. 然後關掉第一個開關，並立即打開第二個開關。

3. 不要動第三個開關，讓它保持關閉。

4. 然後進房間檢查燈。

現在，當我進房間時：

- 如果一盞燈是亮的，那它一定是由第二個開關控制的，因為我剛剛打開了第二個開關。

- 如果一盞燈是滅的但熱的，那它是由第一個開關控制的，因為我打開過它一段時間，然後關掉，所以它還熱。

- 如果一盞燈是滅的但冷的，那它是由第三個開關控制的，因為我從未動過第三個開關，所以它從沒亮過，是冷的。

這樣，我就能區分所有三個開關對應的燈。

我需要確保第一個開關打開的時間足夠長，讓燈泡變熱。通常，幾分鐘就夠了，讓燈泡溫熱。

操作順序：

- 標記開關為 A、B、C，對應燈 X、Y、Z，但我不需要知道燈的名字，只需要找出對應。

- 打開開關 A，等待 5 分鐘（讓燈泡變熱）。

- 關掉開關 A。

- 立即打開開關 B。

- 現在，開關 A 關、開關 B 開、開關 C 關（假設初始狀態都關）。

- 進房間。

檢查燈：

- 亮著的燈：對應開關 B。

- 滅著但熱的燈：對應開關 A。

- 滅著但冷的燈：對應開關 C。

完美。

如果開關初始狀態不明，但問題說我可以隨意操作開關，所以我應該先確保所有開關都在關閉狀態，然後開始實驗。

問題沒有指定初始狀態，所以我最好先關掉所有開關，然後開始。

但關掉所有開關可能不必要，因為我可以直接操作。

為了安全起見，我可以先關掉所有開關，等待幾分鐘，讓燈泡冷卻，然後開始實驗。

但問題沒有說燈泡初始狀態，所以我假設燈泡是冷的。

在標準解法中，我們假設開始時

In [25]:
print(response6["choices"][0]["message"]["content"])

要判斷每個開關對應哪一盞燈，你只能進房間一次，因此需要利用燈泡的狀態（亮、滅但熱、滅但冷）來區分。燈泡在亮過後會變熱，冷卻需要時間，這提供了第三種狀態（滅但熱）。以下是操作步驟：

1. **準備階段（在房間外操作開關）**：
   - 將三個開關標記為開關 A、開關 B 和開關 C（以便區分）。
   - 先確保所有開關都處於「關」的位置，等待約 5 分鐘，讓所有燈泡完全冷卻（這確保初始狀態一致）。

2. **操作開關順序**：
   - 打開開關 A，保持開啟狀態約 5 分鐘（讓對應的燈泡變熱）。
   - 5 分鐘後，關掉開關 A。
   - 立即打開開關 B（不要等待）。
   - 保持開關 C 始終關閉（從未動過）。
   - 操作完成後，立即進房間檢查（避免燈泡冷卻）。

3. **進房間檢查燈泡狀態**：
   - 觀察燈泡的三種狀態：
     - **亮的燈**：對應開關 B（因為開關 B 被打開，燈泡亮著）。
     - **滅但熱的燈**（用手觸摸確認溫度）：對應開關 A（因為開關 A 被打開過一段時間，燈泡變熱，然後關掉）。
     - **滅但冷的燈**：對應開關 C（因為開關 C 從未開啟，燈泡未亮過，保持冷卻）。

### 為什麼這樣有效？
- 開關 B 控制的燈泡是亮的，因為你剛打開開關 B。
- 開關 A 控制的燈泡是滅但熱的，因為它被打開過 5 分鐘，熱量尚未散失。
- 開關 C 控制的燈泡是滅但冷的，因為它從未被開啟，沒有熱量產生。
- 這樣，三種狀態（亮、滅熱、滅冷）正好對應三個開關，讓你只進房間一次就能判斷所有對應關係。

### 注意事項：
- **時間控制**：等待 5 分鐘是為了讓燈泡充分加熱（適用於白熾燈等會發熱的燈泡；如果是 LED 燈，可能不適用，但此謎題通常假設傳統燈泡）。
- **立即進房間**：關掉開關 A 後立即打開開關 B 並進房間，是為了確保開關 A 的燈泡還來不及冷卻。
- **開關標記**：開關必須事先標記（如 A、B、C），以便進房間後能對應燈泡狀態。燈泡在房間內可依位置區分（如左、中、右）。

這個方法可靠且只需進房間一次，就能完整判斷每個開關與燈泡的對應關係。


In [26]:
user_message = "幫我從 1 到 10000 之間隨機選一個整數"

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response7 = get_deepseek_completion(messages)
response7

{'id': 'be28f4ca-a883-4c3a-8804-d845c66e5b26',
 'object': 'chat.completion',
 'created': 1752474110,
 'model': 'deepseek-reasoner',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '以下是從 1 到 10000 之間隨機選取的整數：\n\n**隨機整數：**  \n`{{ random.randint(1, 10000) }}`  \n\n（每次重新整理或執行都會產生一個新的隨機數，範圍包含 1 和 10000）  \n\n### 技術說明：\n- 使用 Python 的 `random` 模組確保均勻分布（每個數字出現機率相同）。\n- 若需要重現結果，可設定隨機種子（seed），但此處為完全隨機。\n\n需要程式碼範例或其他協助嗎？',
    'reasoning_content': '我們需要從1到10000之間隨機選一個整數。在Python中，我們可以使用random模組中的randint函數來達成。\n 注意：範圍包含1和10000。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 23,
  'completion_tokens': 172,
  'total_tokens': 195,
  'prompt_tokens_details': {'cached_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 42},
  'prompt_cache_hit_tokens': 0,
  'prompt_cache_miss_tokens': 23},
 'system_fingerprint': 'fp_393bca965e_prod0623_fp8_kvcache'}

In [28]:
print(response7["choices"][0]["message"]["reasoning_content"])

我們需要從1到10000之間隨機選一個整數。在Python中，我們可以使用random模組中的randint函數來達成。
 注意：範圍包含1和10000。


In [29]:
print(response7["choices"][0]["message"]["content"])

以下是從 1 到 10000 之間隨機選取的整數：

**隨機整數：**  
`{{ random.randint(1, 10000) }}`  

（每次重新整理或執行都會產生一個新的隨機數，範圍包含 1 和 10000）  

### 技術說明：
- 使用 Python 的 `random` 模組確保均勻分布（每個數字出現機率相同）。
- 若需要重現結果，可設定隨機種子（seed），但此處為完全隨機。

需要程式碼範例或其他協助嗎？


## Claude Sonnet 3.7 可以 enable thinking

2025/2/25 推出


In [ ]:
# from google.colab import userdata
# claude_api_key = userdata.get("claude_api_key")

In [30]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
claude_api_key = os.getenv('claude_api_key')


In [31]:
def get_claude_completion(messages, model="claude-3-7-sonnet-20250219", temperature=1, max_tokens = 8000):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens,
              "thinking": {
                "type": "enabled",
                "budget_tokens": 2000
               }
            }

  headers = { "x-api-key": claude_api_key, "anthropic-version": "2023-06-01", "Content-Type": "application/json" }
  response = requests.post('https://api.anthropic.com/v1/messages', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj
  else :
    return obj

In [32]:
user_message = "房間裡有三盞燈外面有三個開關。你站在房間外，看不到燈的狀態。你可以隨意操作開關（開或關），也可以進房間檢查一次燈的情形，但只能進去一次。請問：你該怎麼操作，才能判斷每個開關對應哪一盞燈？"

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_claude_completion(messages)
response

{'id': 'msg_01U28sBvWQgaLaLmb5WTUwBx',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-7-sonnet-20250219',
 'content': [{'type': 'thinking',
   'thinking': '這是一道經典的腦筋急轉彎問題。\n\n我們有3個開關和3盞燈，需要確定哪個開關控制哪盞燈。但是有限制：\n1. 我們一開始站在房間外，看不到燈的狀態\n2. 可以隨意操作開關\n3. 只能進入房間一次來檢查燈的情況\n\n如果我們只能進去一次，那麼我們需要通過某種方式，一次性獲取所有信息。\n\n思考一下，燈有兩種狀態：開和關。但這並不足以區分所有三個開關。\n\n不過，燈泡點亮時會發熱。所以我們可以利用這一點：\n1. 先將第一個開關打開一段時間（例如15分鐘），讓對應的燈泡發熱\n2. 然後關閉第一個開關\n3. 再將第二個開關打開\n4. 然後進入房間\n\n這樣，我們可以通過以下方式判斷：\n- 亮著的燈對應第二個開關\n- 不亮但摸起來是熱的燈對應第一個開關\n- 不亮且摸起來是冷的燈對應第三個開關\n\n這個方法利用了燈泡除了亮/不亮之外還有溫度這個特性，從而能夠一次檢查就獲取所有需要的信息。\n\n讓我用中文清晰地表達這個解決方案。',
   'signature': 'ErUBCkYIBRgCIkA1MvFe+oNsGtJBYPdvk8w2Oa1b+demlPDjSxBDwRTP2HKFcfO1OkJxvGs3attrLbunLyBLVmhTpwJLwOYnLB9pEgxuyXfkH20wjswMfmgaDNkF2KLIj9ECxH2bSSIw7b+B45oF36AmbkybQpq6kfUSLb8qJYR0KcL0i1gQUu6tknn1RU59pdACMEBh5MXEKh184SGr6lPeA1CGXHOhJ1wbeR/jF/3TA8oPQDJTAhgC'},
  {'type': 'text',
   'text': '# 判斷三個開關對應三盞燈的方法\n\n這個問題可以通過利用燈泡的物理特性來解決。我的操作步驟如下：\n\n### 步驟一：操作開關\n1. 打開第一個開關，讓它保持開啟狀態約10-15分鐘（讓

## Gemini 2.5 系列 也是混合 thinking 模型

https://ai.google.dev/gemini-api/docs/thinking?hl=zh-tw

## Cookbook 案例

* 合成數據和驗證數據 Using reasoning for data validation
 https://cookbook.openai.com/examples/o1/using_reasoning_for_data_validation
* 根據場景生成 SOP: Using reasoning for routine generation
 https://cookbook.openai.com/examples/o1/using_reasoning_for_routine_generation